## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
## Part 1

In [17]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [18]:
# retrieve data from URL and create a BeautifulSoup object
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

In [19]:
# apply 'Hints for scraping Notebook'
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')', ' ')).strip(' ')
        table_contents.append(cell)

print(table_contents)
df = pd.DataFrame(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

In [28]:
df['Borough'].value_counts()

North York                                                      24
Downtown Toronto                                                17
Scarborough                                                     17
Etobicoke                                                       11
Central Toronto                                                  9
West Toronto                                                     6
York                                                             5
East Toronto                                                     4
East York                                                        4
MississaugaCanada Post Gateway Processing Centre                 1
East TorontoBusiness reply mail Processing Centre969 Eastern     1
Queen's Park                                                     1
EtobicokeNorthwest                                               1
East YorkEast Toronto                                            1
Downtown TorontoStn A PO Boxes25 The Esplanade                

In [29]:
# change some borough names
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown Toronto', 
                                       'East YorkEast Toronto': 'East Toronto', 
                                       'EtobicokeNorthwest': 'Etobicoke Northwest',
                                       'East TorontoBusiness reply mail Processing Centre969 Eastern': 'East Toronto', 
                                       'MississaugaCanada Post Gateway Processing Centre': 'Mississauga'})

In [30]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [31]:
df.shape

(103, 3)

## Part 2

In [32]:
# get geographical coordinates of each postal code
url = 'https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv'
geodata = pd.read_csv(url)
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
# change column name
geodata = geodata.rename(columns = {'Postal Code': 'PostalCode'})
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
# merge datasets
df2 = pd.merge(df, geodata, on = 'PostalCode')
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [35]:
df2.shape

(103, 5)

## Part 3

In [42]:
!pip install geopy
!pip install folium
from geopy.geocoders import Nominatim
import folium

In [73]:
# work with only borough that contains word Toronto 
df3 = df2[df2['Borough'].str.contains('Toronto')]
df4 = df3.reset_index(drop = True)
df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [74]:
df4['Borough'].value_counts()

Downtown Toronto    18
Central Toronto      9
East Toronto         6
West Toronto         6
Name: Borough, dtype: int64

In [75]:
# use geopy library to get latitude and longitude values of Toronto
address = 'Toronto'
geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [76]:
# create map of Toronto using latitude and longitude values
map_trt = folium.Map(location = [latitude, longitude], zoom_start = 10)

In [77]:
# add markers to map
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_trt)  
    
map_trt

In [78]:
# only to explore dt Toronto
dt_trt = df4[df4['Borough'] == 'Downtown Toronto'].reset_index(drop = True)
dt_trt.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [79]:
# get the geographical coordinates of dt Toronto.
address = 'Downtown Toronto'
geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [81]:
# create map of dt Toronto using latitude and longitude values
map_dttrt = folium.Map(location = [latitude, longitude], zoom_start = 11)

In [82]:
# add markers to map
for lat, lng, label in zip(dt_trt['Latitude'], dt_trt['Longitude'], dt_trt['Neighborhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_dttrt)  
    
map_dttrt

In [87]:
# define Foursquare credentials
CLIENT_ID = 'RHAU3ERYOP3NLJ33QUCOHTIQUUGA5V1U5PL3D0YQGTH5NLXG'
CLIENT_SECRET = 'VKOK31A5ZL13ZVUGE0R1MTPUSMH3XYHZGEOTRPS33PCOZX5D'
VERSION = '20210419'
print('Foursquare Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Foursquare Credentails:
CLIENT_ID: RHAU3ERYOP3NLJ33QUCOHTIQUUGA5V1U5PL3D0YQGTH5NLXG
CLIENT_SECRET:VKOK31A5ZL13ZVUGE0R1MTPUSMH3XYHZGEOTRPS33PCOZX5D


In [86]:
# explore St. James Town eighborhood in dt Toronto
dt_trt.loc[2, 'Neighborhood']
neighborhood_latitude = dt_trt.loc[2, 'Latitude']
neighborhood_longitude = dt_trt.loc[2, 'Longitude']
neighborhood_name = dt_trt.loc[2, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of St. James Town are 43.6514939, -79.3754179.


In [88]:
# get the top 100 venues that are in St. James Town within a radius of 500 meters
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=RHAU3ERYOP3NLJ33QUCOHTIQUUGA5V1U5PL3D0YQGTH5NLXG&client_secret=VKOK31A5ZL13ZVUGE0R1MTPUSMH3XYHZGEOTRPS33PCOZX5D&v=20210419&ll=43.6514939,-79.3754179&radius=500&limit=100'

In [89]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607e32ac69a5f62da79c7cdb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'St. Lawrence',
  'headerFullLocation': 'St. Lawrence, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 79,
  'suggestedBounds': {'ne': {'lat': 43.6559939045, 'lng': -79.36921018606671},
   'sw': {'lat': 43.646993895499996, 'lng': -79.3816256139333}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '583f54320923922e8141d360',
       'name': 'Crepe TO',
       'location': {'address': '52 Church Street',
        'lat': 43.650063279511016,
        'lng': -79.37458664523871,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.650063279

In [90]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [92]:
# clean the json and structure it into a pandas dataframe
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']   
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

<ipython-input-92-db215c0c3d37>:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,Crepe TO,Creperie,43.650063,-79.374587
1,Fahrenheit Coffee,Coffee Shop,43.652384,-79.372719
2,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
3,GEORGE Restaurant,Restaurant,43.653346,-79.374445
4,Hogtown Smoke,Food Truck,43.649287,-79.374689


In [93]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

79 venues were returned by Foursquare.


In [94]:
# create a function to repeat the same process to all the neighborhoods in dt Toronto
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [95]:
# run the above function on each neighborhood and create a new dataframe called dttrt_venues
dttrt_venues = getNearbyVenues(names = dt_trt['Neighborhood'],
                                   latitudes = dt_trt['Latitude'],
                                   longitudes = dt_trt['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [109]:
dttrt_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [101]:
dttrt_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,62,62,62,62,62,62
Christie,15,15,15,15,15,15
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Enclave of M5E,97,97,97,97,97,97
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100


In [102]:
print('There are {} uniques categories.'.format(len(dttrt_venues['Venue Category'].unique())))

There are 205 uniques categories.


In [103]:
# analyze each neighborhood
dttrt_onehot = pd.get_dummies(dttrt_venues[['Venue Category']], prefix = '', prefix_sep = '')
dttrt_onehot['Neighborhood'] = dttrt_venues['Neighborhood'] 
fixed_columns = [dttrt_onehot.columns[-1]] + list(dttrt_onehot.columns[:-1])
dttrt_onehot = dttrt_onehot[fixed_columns]
dttrt_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
dttrt_onehot.shape

(1185, 205)

In [108]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
dttrt_grouped = dttrt_onehot.groupby('Neighborhood').mean().reset_index()
dttrt_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.016949,0.000000,0.000000,0.0,0.0,0.016949,0.0,0.0,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
2,Central Bay Street,0.016129,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.016129,0.000000,0.000000,0.0,0.0,0.016129,0.0,0.0,0.016129,0.000000
3,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
4,Church and Wellesley,0.027397,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.013699,0.0,...,0.013699,0.013699,0.013699,0.0,0.0,0.000000,0.0,0.0,0.000000,0.013699


In [112]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5
for hood in dttrt_grouped['Neighborhood']:
    print('----' + hood + '----')
    temp = dttrt_grouped[dttrt_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0   Coffee Shop  0.08
1  Cocktail Bar  0.05
2    Restaurant  0.03
3      Pharmacy  0.03
4      Beer Bar  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge  0.12
2    Airport Terminal  0.12
3  Airport Food Court  0.06
4        Airport Gate  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
                venue  freq
0       Grocery Store  0.27
1                Café  0.20
2                Park  0.13
3          Restaurant  0.07
4  Italian Restaurant  0.07


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.07
1      Sushi Restaurant  0.07
2   Japanese Restaurant  0.07
3            Restaurant  0.0

In [111]:
# write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [114]:
# create the new dataframe and display the top 10 venues for each neighborhood
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = dttrt_grouped['Neighborhood']

for ind in np.arange(dttrt_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dttrt_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Pharmacy,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Jazz Club
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Rental Car Location,Boat or Ferry,Bar,Harbor / Marina,Airport Gate
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Gym,Miscellaneous Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Nightclub,Restaurant,Italian Restaurant,Baby Store,Candy Store,Coffee Shop,Creperie
4,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Restaurant,Yoga Studio,Pub,Gay Bar,Hotel,Fast Food Restaurant,Mediterranean Restaurant


In [119]:
# run k-means to cluster the neighborhood into 5 clusters
from sklearn.cluster import KMeans
kclusters = 5
dttrt_grouped_clustering = dttrt_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(dttrt_grouped_clustering)
kmeans.labels_[0:10] 

array([0, 3, 0, 1, 0, 0, 0, 0, 0, 0])

In [121]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
dt_trt = dt_trt.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on = 'Neighborhood')
dt_trt.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Restaurant,Pub,Theater,Cosmetics Shop,Brewery
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Lingerie Store,Theater,Italian Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Beer Bar,Farmers Market,Hotel,Italian Restaurant,Lingerie Store
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Pharmacy,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Jazz Club
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Gym,Miscellaneous Shop,Japanese Restaurant


In [123]:
# visualize the resulting clusters
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(dt_trt['Latitude'], dt_trt['Longitude'], dt_trt['Neighborhood'], dt_trt['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [124]:
# examine clusters
dt_trt.loc[dt_trt['Cluster Labels'] == 0, dt_trt.columns[[1] + list(range(5, dt_trt.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Restaurant,Pub,Theater,Cosmetics Shop,Brewery
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Bubble Tea Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Lingerie Store,Theater,Italian Restaurant
2,Downtown Toronto,0,Coffee Shop,Café,Cosmetics Shop,Cocktail Bar,Gym,Beer Bar,Farmers Market,Hotel,Italian Restaurant,Lingerie Store
3,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Pharmacy,Beer Bar,Farmers Market,Bakery,Restaurant,Cheese Shop,Seafood Restaurant,Jazz Club
4,Downtown Toronto,0,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Gym,Miscellaneous Shop,Japanese Restaurant
6,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Deli / Bodega,Thai Restaurant,Gym,Clothing Store,Concert Hall,Salad Place,American Restaurant
7,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Scenic Lookout,Sporting Goods Shop,Fried Chicken Joint,History Museum
8,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Salad Place,Seafood Restaurant,Japanese Restaurant,Italian Restaurant,Sushi Restaurant,Breakfast Spot
9,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Deli / Bodega,Japanese Restaurant,American Restaurant,Seafood Restaurant
14,Downtown Toronto,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Restaurant,Beer Bar,Hotel,Café,Gym


In [125]:
dt_trt.loc[dt_trt['Cluster Labels'] == 1, dt_trt.columns[[1] + list(range(5, dt_trt.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,1,Grocery Store,Café,Park,Nightclub,Restaurant,Italian Restaurant,Baby Store,Candy Store,Coffee Shop,Creperie


In [126]:
dt_trt.loc[dt_trt['Cluster Labels'] == 2, dt_trt.columns[[1] + list(range(5, dt_trt.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Park,Playground,Trail,Dessert Shop,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [127]:
dt_trt.loc[dt_trt['Cluster Labels'] == 3, dt_trt.columns[[1] + list(range(5, dt_trt.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Rental Car Location,Boat or Ferry,Bar,Harbor / Marina,Airport Gate


In [128]:
dt_trt.loc[dt_trt['Cluster Labels'] == 4, dt_trt.columns[[1] + list(range(5, dt_trt.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,4,Café,Bar,Bakery,Bookstore,Sandwich Place,Japanese Restaurant,Yoga Studio,Sushi Restaurant,Beer Bar,Bank
11,Downtown Toronto,4,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Burger Joint,Bakery,Grocery Store,Bar,Arts & Crafts Store


## Thanks!